In [50]:
import numpy as np
import pandas as pd
import json
import gc
from collections import defaultdict
import sys
from enum import Enum
import random
import pickle

random.seed(824)

print("Setup complete!")

Setup complete!


In [51]:
ml_ratings_file = "rating_features1.csv"
ratings_df = pd.read_csv(ml_ratings_file)
ratings_df

,Unnamed: 0,userId,movieId,rating
0,0,1,296,5.0
1,1,1,306,3.5
2,2,1,307,5.0
3,3,1,665,5.0
4,4,1,899,3.5
...,...,...,...,...
24859479,25000090,162541,50872,4.5
24859480,25000091,162541,55768,2.5
24859481,25000092,162541,56176,2.0
24859482,25000093,162541,58559,4.0


In [52]:
users = set(ratings_df["userId"])
num_of_users = len(users)
num_of_users

162541

In [45]:
user_rating_counts = ratings_df["userId"].value_counts().rename_axis('userId').reset_index(name='counts')
user_rating_counts

,userId,counts
0,72315,19822
1,80974,8566
2,137293,8109
3,33844,7412
4,20055,7352
...,...,...
162536,75043,15
162537,121732,15
162538,122860,14
162539,84947,12


In [47]:
valid = True
def check_valid_users(row):
    if row["counts"] < 5:
        print(row["userId"])
        valid = False
user_rating_counts.apply(check_valid_users, axis=1)
print(valid)

True


In [53]:

class Split(Enum):
    TRAIN = 1
    VALID = 2
    TEST = 3

def build_edges(row):
    user_id = int(row["userId"])
    movie_id = int(row["movieId"])
    user = user_id - 1
    movie = movie_map[movie_id]
    rating = row["rating"]
    split = random.choices(list(Split), weights=(70,10,20), k=1)[0].value
    movie_adj_list_test[movie].add((user, rating))
    user_adj_list_test[user].add((movie, rating))
    if split == Split.TEST.value:
        edge_list_test.append((user, movie, rating))
    elif split == Split.VALID.value:
        movie_adj_list_valid[movie].add((user, rating))
        user_adj_list_valid[user].add((movie, rating))
        edge_list_valid.append((user, movie, rating))
    elif split == Split.TRAIN.value:
        movie_adj_list_valid[movie].add((user, rating))
        user_adj_list_valid[user].add((movie, rating))
        movie_adj_list_train[movie].add((user, rating))
        user_adj_list_train[user].add((movie, rating))
        edge_list_train.append((user, movie, rating))
    

In [54]:
ml_movies_file = "movie_features1.csv"
movies_df = pd.read_csv(ml_movies_file)

user_feat = []
movie_map = {} # map movieId to position in movies_feat

movies_feat = movies_df.iloc[:, 1:].to_numpy().tolist()

i = 0
for idx in movies_df.movieId:
	movie_map[int(idx)] = i
	i += 1

del movies_df
gc.collect()

num_of_movies = len(movies_feat)

#ml_ratings_file = "rating_features1.csv"
#ratings_df = pd.read_csv(ml_ratings_file)

movie_adj_list_train = [set() for _ in range(num_of_movies)]
movie_adj_list_valid = [set() for _ in range(num_of_movies)]
movie_adj_list_test = [set() for _ in range(num_of_movies)]
user_adj_list_train = [set() for _ in range(num_of_users)]
user_adj_list_valid = [set() for _ in range(num_of_users)]
user_adj_list_test = [set() for _ in range(num_of_users)]
edge_list_train = []
edge_list_valid = []
edge_list_test = []

ratings_df.apply(build_edges, axis=1)

del ratings_df
gc.collect()



0

In [55]:
print(len(movies_feat))
print(len(movies_feat[0]))
print(len(movie_adj_list_train))
print(len(movie_adj_list_valid))
print(len(movie_adj_list_test))
print(len(user_adj_list_train))
print(len(user_adj_list_valid))
print(len(user_adj_list_test))
print(len(edge_list_train))
print(len(edge_list_valid))
print(len(edge_list_test))

27832
5511
27832
27832
27832
162541
162541
162541
17402244
2485517
4971723


In [68]:
def find_in_valid(movie):
    j = 0
    user = None
    rating = None
    for edge in edge_list_valid:
        if movie == edge[1]:
            user = edge[0]
            rating = edge[2]
            break
        j += 1
    return (user, movie, rating)

def find_in_test(movie):
    j = 0
    user = None
    rating = None
    for edge in edge_list_test:
        if movie == edge[1]:
            user = edge[0]
            rating = edge[2]
            break
        j += 1
    return (user, movie, rating)

def update_valid(el):
    edge_list_valid.remove(el)
    edge_list_train.append(el)
    movie_adj_list_train[el[1]].add((el[0], el[2]))
    user_adj_list_train[el[0]].add((el[1], el[2]))

def update_test(el):
    edge_list_test.remove(el)
    edge_list_train.append(el)
    movie_adj_list_train[el[1]].add((el[0], el[2]))
    user_adj_list_train[el[0]].add((el[1], el[2]))

neg_idx = []
i = 0
for neigh in movie_adj_list_train:
    if len(neigh) < 2:
        neg_idx.append(i)
    i += 1
print(neg_idx)


[]


In [67]:
for idx in neg_idx:
    print("For idx")
    edge1 = find_in_test(idx)
    print(f"Edge 1: {edge1}")
    update_test(edge1)
    if len(movie_adj_list_train[idx]) < 2:
        edge2 = find_in_valid(idx)
        print(f"Edge 2: {edge2}")
        update_valid(edge2)

For idx
Edge 1: (49973, 16977, 3.0)
For idx
Edge 1: (39741, 17096, 1.5)
For idx
Edge 1: (19462, 17885, 3.0)
For idx
Edge 1: (14538, 18082, 2.5)
For idx
Edge 1: (72314, 18427, 3.5)
For idx
Edge 1: (12856, 18821, 1.5)
For idx
Edge 1: (11756, 18897, 3.5)
For idx
Edge 1: (65771, 19657, 3.5)
For idx
Edge 1: (12263, 19767, 4.0)
For idx
Edge 1: (39741, 19805, 2.5)
For idx
Edge 1: (42406, 19837, 4.0)
For idx
Edge 1: (24609, 20107, 3.5)
For idx
Edge 1: (22174, 20485, 4.0)
For idx
Edge 1: (30878, 20792, 3.5)
For idx
Edge 1: (42485, 20969, 4.0)
For idx
Edge 1: (34691, 21804, 2.5)
For idx
Edge 1: (2083, 21968, 4.5)
For idx
Edge 1: (7864, 22490, 1.0)
For idx
Edge 1: (72314, 22834, 2.5)
For idx
Edge 1: (78333, 22943, 3.0)
For idx
Edge 1: (116134, 22980, 0.5)
For idx
Edge 1: (14538, 23038, 4.5)
Edge 2: (67361, 23038, 5.0)
For idx
Edge 1: (69810, 23484, 0.5)
Edge 2: (19462, 23484, 2.5)
For idx
Edge 1: (23762, 23864, 3.5)
For idx
Edge 1: (79192, 24250, 2.5)
For idx
Edge 1: (24868, 24597, 4.0)
For idx
E

In [69]:
def find_in_valid(user):
    j = 0
    user = None
    rating = None
    for edge in edge_list_valid:
        if movie == edge[1]:
            user = edge[0]
            rating = edge[2]
            break
        j += 1
    return (user, movie, rating)

def find_in_test(user):
    j = 0
    user = None
    rating = None
    for edge in edge_list_test:
        if movie == edge[1]:
            user = edge[0]
            rating = edge[2]
            break
        j += 1
    return (user, movie, rating)

neg_idx = []
i = 0
for neigh in user_adj_list_train:
    if len(neigh) < 2:
        neg_idx.append(i)
    i += 1
print(neg_idx)

[]


In [40]:
train = 0
valid = 0
test = 0
for i in range(100):
    split = random.choices(list(Split), weights=(70,10,20), k=1)[0].value
    if split == Split.TRAIN.value:
       train += 1
    elif split == Split.VALID.value:
        valid += 1
    elif split == Split.TEST.value:
        test += 1
print(f"train: {train}")
print(f"valid: {valid}")
print(f"test: {test}")

train: 71
valid: 8
test: 21


In [70]:
data = dict()
data["movies_feat"] = movies_feat
data["movie_map"] = movie_map
data["movie_adj_list_train"] = movie_adj_list_train
data["movie_adj_list_valid"] = movie_adj_list_valid
data["movie_adj_list_test"] = movie_adj_list_test
data["user_adj_list_train"] = user_adj_list_train
data["user_adj_list_valid"] = user_adj_list_valid
data["user_adj_list_test"] = user_adj_list_test
data["edge_list_train"] = edge_list_train
data["edge_list_valid"] = edge_list_valid
data["edge_list_test"] = edge_list_test

a_file = open("prepared_data/data1.pkl", "wb")
pickle.dump(data, a_file)
a_file.close()


In [8]:
a_file = open("prepared_data/data.pkl", "rb")
data = pickle.load(a_file)
a_file.close()
movies_feat = data["movies_feat"]
movie_map = data["movie_map"]
movie_adj_list = data["movie_adj_list"]
user_adj_list = data["user_adj_list"]
edge_list = data["edge_list"]

In [15]:
print(len(user_adj_list))

162541


In [6]:
list(Split)

[<Split.TRAIN: 1>, <Split.VALID: 2>, <Split.TEST: 3>]

In [4]:
dictionary_data = {"a": [4,5], "b": {"g": [1,7]}}

a_file = open("prepared_data/data.pkl", "wb")
pickle.dump(dictionary_data, a_file)
a_file.close()

a_file = open("prepared_data/data.pkl", "rb")
output = pickle.load(a_file)
print(output)
a_file.close()

{'a': [4, 5], 'b': {'g': [1, 7]}}
